https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb

In [386]:
import torch
from torch import nn
import torch.nn.functional as F

import re
import numpy as np

In [563]:
with open('./drake_archive/drake_lyrics.txt') as f:
    text_dataset_raw = f.read()


In [564]:
text_dataset = re.findall('...............',text_dataset_raw)

text_dataset[:10]

['Put my feelings',
 'Always been a g',
 'Certified lover',
 ' boy, somehow s',
 'Heart is only g',
 "Someone's in my",
 ' ear from the o',
 "Tellin' me that",
 ' I should pay y',
 'Wanted you to n']

In [565]:
# text = ['hey how are you','good i am fine','have a nice day']
text = text_dataset

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [566]:
maxlen = len(max(text, key=len))

for i in range(len(text)):
    while len(text[i]) < maxlen:
        text[i] += ' '

text

['Put my feelings',
 'Always been a g',
 'Certified lover',
 ' boy, somehow s',
 'Heart is only g',
 "Someone's in my",
 ' ear from the o',
 "Tellin' me that",
 ' I should pay y',
 'Wanted you to n',
 'ot be with me a',
 'Wanted you to n',
 'ot stay with me',
 'I know, you kno',
 'w, who that per',
 "Doesn't really ",
 "I know you're s",
 'cared of dating',
 ', falling for m',
 'Shorty, surely ',
 'Right here for ',
 'You know, I don',
 'Right here for ',
 "You know I don'",
 'Right here for ',
 'In mind you mak',
 'e me want to do',
 ' things, love y',
 "Like I'm suppos",
 'You make me wan',
 "Like I'm suppos",
 'You make me wan',
 "Like I'm suppos",
 'ed to, remind y',
 "I know you're s",
 'cared of dating',
 ', falling for m',
 'Shorty, by now ',
 'Right here for ',
 'You know, I don',
 'Right here for ',
 "You know I don'",
 'Right here for ',
 'Swishahouse, ba',
 'Wassup to all t',
 'he ladies on th',
 'e Northside, So',
 'uthside, Eastsi',
 'Wassup\u2005in\u2005Bay C',
 'Wassup t

In [567]:
input_seqs = []
target_seqs = []

for sentence in text:
    input_seqs.append(sentence[:-1])
    target_seqs.append(sentence[1:])


In [581]:
train_data_set = []

for sentence in text:
    train_data_set.append((sentence[:-1],sentence[1:]))

train_data_set[5000:5010]

[('it would chang', 't would change'),
 (" but I don't r", "but I don't re"),
 ('ally see no ch', 'lly see no cha'),
 ('nge in us, oh,', 'ge in us, oh, '),
 ('[Chorus: Drake', 'Chorus: Drake]'),
 ('Tat my name on', 'at my name on '),
 ('you so I know ', 'ou so I know i'),
 ("Tat my fuckin'", "at my fuckin' "),
 ('name on you so', 'ame on you so '),
 ("I know it's re", " know it's rea")]

In [641]:
for i in range(len(train_data_set)):
    train_data_set[i] = ([char2int[char] for char in train_data_set[i][0]], [char2int[char] for char in train_data_set[i][1]])

train_data_set[:10]

[([80, 69, 78, 14, 75, 83, 14, 59, 65, 65, 46, 73, 91, 90],
  [69, 78, 14, 75, 83, 14, 59, 65, 65, 46, 73, 91, 90, 57]),
 ([66, 46, 16, 18, 83, 57, 14, 36, 65, 65, 91, 14, 18, 14],
  [46, 16, 18, 83, 57, 14, 36, 65, 65, 91, 14, 18, 14, 90]),
 ([47, 65, 99, 78, 73, 59, 73, 65, 13, 14, 46, 53, 31, 65],
  [65, 99, 78, 73, 59, 73, 65, 13, 14, 46, 53, 31, 65, 99]),
 ([14, 36, 53, 83, 97, 14, 57, 53, 75, 65, 81, 53, 16, 14],
  [36, 53, 83, 97, 14, 57, 53, 75, 65, 81, 53, 16, 14, 57]),
 ([61, 65, 18, 99, 78, 14, 73, 57, 14, 53, 91, 46, 83, 14],
  [65, 18, 99, 78, 14, 73, 57, 14, 53, 91, 46, 83, 14, 90]),
 ([67, 53, 75, 65, 53, 91, 65, 98, 57, 14, 73, 91, 14, 75],
  [53, 75, 65, 53, 91, 65, 98, 57, 14, 73, 91, 14, 75, 83]),
 ([14, 65, 18, 99, 14, 59, 99, 53, 75, 14, 78, 81, 65, 14],
  [65, 18, 99, 14, 59, 99, 53, 75, 14, 78, 81, 65, 14, 53]),
 ([88, 65, 46, 46, 73, 91, 98, 14, 75, 65, 14, 78, 81, 18],
  [65, 46, 46, 73, 91, 98, 14, 75, 65, 14, 78, 81, 18, 78]),
 ([14, 45, 14, 57, 81, 53, 69, 4

# Set batch size when using real dataset

In [637]:
dict_size = len(char2int)
seq_len = maxlen - 1
#batch_size = len(text)
batch_size = 8

def one_hot_encode(seq, dict_size, seq_len):
    features = np.zeros((seq_len, dict_size), dtype=np.float32)
    for i in range(seq_len):
        features[i,seq[i]] = 1
    
    return features


In [640]:
train_data_set[5012][0]

"'Cause when I'"

In [642]:
#experimental hot encode output as well
hot_encoded_train = [(one_hot_encode(train_data_set[i][0], dict_size, seq_len), one_hot_encode(train_data_set[i][1], dict_size, seq_len)) for i in range(len(train_data_set))]

In [643]:
len(hot_encoded_train)

41069

In [644]:
target_batched = torch.utils.data.DataLoader(hot_encoded_train[5000:15000], batch_size=batch_size, shuffle=True) 

In [252]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device('gpu')
else:
    device = torch.device('cpu')

In [645]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = F.softmax(out, dim=1)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [646]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=30, n_layers=5)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters

lr=0.001

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [647]:
n_epochs = 30
# Training Run
for epoch in range(1, n_epochs + 1):
    batch_index = 0
    for data in target_batched:
        batch_index += 1
        x , y = data

        x = torch.Tensor(x)
        y = torch.Tensor(y)

        # use .stack for not hot encoded output

        input_seq = x.to(device)
        target_seq = y.to(device)        

        optimizer.zero_grad() # Clears existing gradients from previous epoch

        output, hidden = model(input_seq)
        output = output.to(device)

        loss = criterion(output.view(-1), target_seq.view(-1))
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
        
    # if epoch % 10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

Epoch: 1/30............. Loss: 0.0080
Epoch: 2/30............. Loss: 0.0077
Epoch: 3/30............. Loss: 0.0076
Epoch: 4/30............. Loss: 0.0077
Epoch: 5/30............. Loss: 0.0072
Epoch: 6/30............. Loss: 0.0067
Epoch: 7/30............. Loss: 0.0068
Epoch: 8/30............. Loss: 0.0067
Epoch: 9/30............. Loss: 0.0072
Epoch: 10/30............. Loss: 0.0071
Epoch: 11/30............. Loss: 0.0066
Epoch: 12/30............. Loss: 0.0063
Epoch: 13/30............. Loss: 0.0066
Epoch: 14/30............. Loss: 0.0058
Epoch: 15/30............. Loss: 0.0057
Epoch: 16/30............. Loss: 0.0071
Epoch: 17/30............. Loss: 0.0073
Epoch: 18/30............. Loss: 0.0061
Epoch: 19/30............. Loss: 0.0056
Epoch: 20/30............. Loss: 0.0067
Epoch: 21/30............. Loss: 0.0060
Epoch: 22/30............. Loss: 0.0061
Epoch: 23/30............. Loss: 0.0067
Epoch: 24/30............. Loss: 0.0073
Epoch: 25/30............. Loss: 0.0062
Epoch: 26/30............. Loss: 0.

In [455]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([char2int[c] for c in character])
    character = one_hot_encode(character, dict_size, character.shape[0])
    character = torch.tensor([character])
    character = character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [312]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    print("chars", chars)
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [659]:
sample(model, 40, start= 'he')

chars ['h', 'e']


'he got a lot the same that I wath the sa'